## Блок 8. Подсчет ROI

In [46]:
# Преамбула
import os
from typing import Callable
from tabulate import tabulate

path_data = "../data"
data_txt = f"{path_data}/data_no_header.txt"

# константы для задачи о типах покупак
SOURCE_TYPE = "sourceType"
ORDER_TYPE = "orderType"

# константы для типов партнёрских программ
CPA_PARTNERS = "cpa-partners"
CPC_PARTNERS = "cpc-partners"
OTHER_PARTNERS = "fixed"

# константы для подчёта ROI 
PROFIT = "profit"
EXPENSE = "expense"
ROI = "roi"

# индексы заголовок таблицы
ID = 0
DATE = 1
USER_ID = 2
DURATION = 3
MEDIUM = 4
SOURCE = 5
COST = 6
ORDER_ID = 7
AMOUNT_PAID = 8

# название заголовок таблиц
headers = ("id", "date", "user_id",	"duration",	"medium", "source", "cost", "order_id", "amount_paid")

# комисия партёрской программы 
commission = {
    CPA_PARTNERS: {
        "burgerclub": 0.3,
        "food-delivery": 0.25,
    },
    CPC_PARTNERS: {
        "city-magazine": 7.0,
        "foody": 9.0
    },
    OTHER_PARTNERS: 4.0
}

## Вспомогательные фукнции 

In [108]:
def print_results(roi_stats):
    """
     Печатает результат подсчёта ROI 
     
    :param roi_stats: 
    :return: 
    """
    headers = ["Name", f"{ROI}", f"{PROFIT}", f"{EXPENSE}"]
    tabular_data = [
        [f"{name}", f"{data[ROI]:.2f}", f"{data[PROFIT]:.2f}", f"{data[EXPENSE]:.2f}"] for name, data in roi_stats.items()]
    table = tabulate(tabular_data=tabular_data, headers=headers, tablefmt='orgtbl')
    print(table)

In [109]:
def conv_float(value: str) -> float:
    """
    Конвертирует строку в число с плавующей запятой
    
    :param value: 
    :return: 
    """
    return float(value.replace(',', '.'))

In [115]:
def unit_uniq_orders(total: dict, current: dict) -> dict:
    """
    Фукцния коллеционирует результаты подсчёта
    
    :param total: 
    :param current: 
    :return: новый словарь с добавленными данными.
    """
    _total = total.copy()
    for key, value in current.items():
        _total.update({key: _total.get(key, 0.0) + float(value)})
    return _total

In [123]:
def costs_classification(amount_paid: float, source: str) -> float:
    """
     Функция подсчитывает затраты с учётом комисии
    :param amount_paid: 
    :param source: 
    :return:  застраты с учётом комисии.
    """
    if source in commission[CPA_PARTNERS]:
            return float(amount_paid*commission[CPA_PARTNERS][source])
    if source in commission[CPC_PARTNERS]:
            return float(commission[CPC_PARTNERS][source])
    else:
        return float(commission[OTHER_PARTNERS])

In [124]:
def source_classification(source: str, medium: str) -> str:
    """
     Типы источника определяются следующим образом
    - если source равен 'google' или 'yandex', то проверяем medium:

        - для medium 'seo' или 'sem' тип источника равен 'search engines seo'

        - для medium 'brand' - тип источника равен 'search engines brand'

        - для остальных случаев тип источника равен 'search engines undefined'

    - если условие не выполнено, то тип источника равен 'other'
    
    :param source: 
    :param medium: 
    :return: тип источника
    """
    if source in ["google", "yandex"]:
        if medium in ["seo", "sem"]:
            return "search engines seo"
        elif medium in ["brand"]:
            return "search engines brand"
        else:
            return "search engines undefined"
    else:
        return "other"

In [125]:
# проверка
costs_classification(10.0, 'google')

4.0

In [126]:
# проверка
source_classification("yandex", "brand")

'search engines brand'

In [127]:
def profit_expense(line: list) -> dict:
    """
     Считаем прибыль от рекламной компании.
     
    :param line: 
    :param partners: 
    :return: возращает словарь ввида : имя_партнёрской_компании -> {прибыль , затраты }.
    """
    stats: dict = {}
    medium: str = line[MEDIUM]
    source: str = line[SOURCE]
    cost: float = conv_float(line[COST])
    amount_paid: float = conv_float(line[AMOUNT_PAID])
    partner_commission: float = costs_classification(amount_paid=amount_paid, source=source)
    name_partner: str = source_classification(source, medium)
    
    stats.update({
        name_partner: {
            PROFIT: amount_paid,
            EXPENSE: cost + partner_commission
        }})
    return stats

In [128]:
def unit_profit_expense(total: dict, current: dict) -> dict:
    """
    Фукцния коллеционирует результаты подсчёта
    
    :param total: 
    :param current: 
    :return: новый словарь с добавленными данными.
    """
    
    _total = total.copy()
    for key, value in current.items():
        _total.setdefault(key, {PROFIT: 0, EXPENSE: 0})
        _total.update({
            key: {
                PROFIT: _total[key].get(PROFIT, 0) + value.get(PROFIT),
                EXPENSE: _total[key].get(EXPENSE, 0) + value.get(EXPENSE)
            }})
    return _total

In [129]:
def calc_roi(total_stats: dict) -> dict:
    """
    Расчитывает ROI по каждой компании
    :param total_stats: 
    :return: 
    """
    _total_stats = total_stats.copy()
    for prog, stats in _total_stats.items():
        roi: float = (stats.get(PROFIT) - stats.get(EXPENSE)) / stats.get(EXPENSE)
        _total_stats[prog].update({
            ROI: roi
        })
    return _total_stats

In [130]:
def get_other_order(line: list) -> dict:
    """
     Фильтрует заказы CPA и CPC партнеров
     
    :param line: 
    :return: 
    """
    order_id = line[ORDER_ID]
    medium = line[MEDIUM]
    if medium == CPC_PARTNERS or medium == CPA_PARTNERS:
        return {} 
    return {order_id: 1}

In [131]:
# основная функция обхода данных
def proc_stats(
        filename: str, 
        calc_stats: Callable[[dict, dict], dict], 
        condition: Callable[..., dict],
        skip_line=False,
) -> dict:
    """
    функция обходит таблицу и делает по каждой строку расчёты. Затем объединяет результат.
    
    :param filename: 
    :param condition: 
    :param skip_line: Если нужно пропустить первую строку, то следует выставить параметр как True
    :return: 
    """
    
    total_stats: dict = {}
    with open(os.path.abspath(filename), "r") as file:
        if skip_line is True:
            file.readline()
        for line in file:
            line = line.strip().split('\t')
            total_stats = calc_stats(total_stats, condition(line))
    return total_stats

## Выполнение основных подсчётов

In [132]:
# подсчёт уникальных заказов
total_uniq_orders = proc_stats(data_txt, unit_uniq_orders, get_other_order)
len(total_uniq_orders)

266

In [133]:
# подсчитать доходы и расходы по каждой компаниям
total_stats = proc_stats(data_txt, unit_profit_expense, profit_expense)
# подсчитать ROI 
roi_stats = calc_roi(total_stats)
print_results(roi_stats)

| Name                 |   roi |   profit |   expense |
|----------------------+-------+----------+-----------|
| search engines seo   |  3.23 |   2902.4 |    686.27 |
| other                |  2.68 |   1992.8 |    541.11 |
| search engines brand |  3.51 |    234.6 |     52    |


In [134]:
# найти истоничник с максимимальным значением ROI
last_roi = ("", -1)
for source, data in roi_stats.items():
    last_roi = max((source, data[ROI]), last_roi, key=lambda x: x[1])
    
print(f"{last_roi[0]} {last_roi[1]:.2f}")

search engines brand 3.51
